In [19]:
from dotenv import load_dotenv
import os
load_dotenv() 

True

In [20]:
from langchain_anthropic import ChatAnthropic
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

In [21]:
model = ChatAnthropic(model='claude-3-opus-20240229')

In [22]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("HUGGINGFACE_API_KEY"), model_name="allenai/longformer-base-40962"
)